<a href="https://colab.research.google.com/github/The-allison-k/Gemini-Notebooks/blob/main/Recipe_Replicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Code for Recipe Replicator**

This notebook uses Google Search as a tool, provides Gemini with an Image from the web, and creates a multi-turn chat. The prompting is having it re-create a recipe from a restaurant in NYC. You can change the prompting and image input to try to re-create recipes from other places.

**Are you a startup building on Google Cloud?**

We'd love to connect and explore how we can support your journey. Learn more about our Startups program [here](https://startup.google.com/)

Reach out to me at build_with_allison@google.com

## **Install, Auth, and Imports**

In [ ]:
%pip install --upgrade --quiet google-genai

import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()


from IPython.display import HTML, Image, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    SafetySetting,
    Tool,
    ToolCodeExecution,
)



## **Set the project and models to be used**
 **Use a Google Cloud Project:** this requires enabling the Vertex AI API in your Google Cloud project.
    [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)


**Dont have a project?** Follow [these](https://developers.google.com/workspace/guides/create-project) instructions

In [ ]:
import os

PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)


MODEL_ID = "gemini-2.5-pro-preview-06-05"  # @param {type: "string"}


## **Add Google search as a tool**
* Learn more about [Grounding with Google Search ](https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/grounding-with-google-search)

* Learn more about [Grounding](https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview)



In [ ]:
google_search_tool = Tool(google_search=GoogleSearch())

## **Create a Multi-Turn Chat with Google Search as a tool**
Using a multi-turn chat allows you to refine the recipe with the model, asking follow-up questions and providing feedback based on its initial suggestions. This interactive process helps in getting a more accurate and tailored recipe.

In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(tools=[google_search_tool])
    )


## **Link it to an image from a Google review**
This image of the dish from a Google review will be used by the model to visually analyze the dish and inform the recipe recreation. The model can identify ingredients, textures, and plating styles from the image.

In [ ]:
image_file_url = "https://lh3.googleusercontent.com/gps-cs-s/AC9h4nqrsiS_gziCB08g10yO5SOKG_tM90C-YL-iKWT_Rg7Cyxr1RbwN9lNpEUf3Bj7ejXk-GQpYuQ5YvcJ_HVT-e3z_gjPVd04GPD11bPA95Qgx89atXn4ODmaMcVqShzCnJpoboNBn=s1360-w1360-h1020-rw" # @param {type: "string", placeholder: "[URL]", isTemplate: true}

display(Image(url=image_file_url, width=400))

## **Write your prompt**

In [ ]:
Prompt = "You are a professional vegan chef. You are trying to re-create a recipe from a restaurant you went to called Avant Garden in NYC. This is the full restaurant menu:    C O L D  BABY GEM (NF)  charred gem lettuce. caesar. roasted tomato. avocado. garlic nori crouton.  $18  BRUSSELS  crispy brussels. endive. arugula. fig gastrique. candied walnuts.  $19  AVOCADO (GF)  carrot ginger dressing. crispy rice. miso glaze. blistered shishito  $22  T O A S T  FIG  soy ricotta. balsamic pearls. pistachio. white balsamic reduction. basil.  $20  ARTICHOKE  spinach artichoke puree. black truffle vinaigrette. jicama. cashew.  $21  CREMINI MUSHROOM  grape. onion marmalade. herbs de provence. walnut purée.  $19  SUNCHOKE  pickled pear. pistachio. mint. olive purée.  $18  H O T  SQUASH SOUP  kambucha. corn & butternut squash. cashew crema.  $18  SCORCHED CAULIFLOWER (GF, NF)  toasted garlic. fried capers. cauliflower puree.  $26  CELERY ROOT PASTA (NF)  shaved black truffle. shiitake velour. gremolata  $29  HEN OF THE WOODS (GF, NF)  mushroom puree. kohlrabi. pickled hon shimeji.  $21  ROASTED CARROT (NF)  farro. ramp chimichurri. cucumber tzatziki.  $26  RISOTTO (GF, NF)  lobster mushroom. watercress. balsamic pearls. basil oil.  $28  SPAGHETTI (NF)  house-made pasta. carbonara sauce. smoked mushrooms.  $24  TAMAL  mole. bean & epazote sour cream. jackfruit. pickled red onion  $26  PAELLA (GF)  calasparra rice. royal trumpet. almond. red pepper & merguez aioli  $29    I really enjoyed this dish: SCORCHED CAULIFLOWER (GF, NF)  toasted garlic. fried capers. cauliflower puree.  I need you to make me a recipe to for me to make it. The restaurant told me that there is Calabrian chili oil. you have a picture of the food. Use google to search reviews to get an idea of the flavor. Ask any questions needed then write me a full recipe to make this dish. Keep in mind it is vegan. Use google search to look at reviews and find other similar recipes. Be sure to look into vegan recipes and how to replicate the scorched cauliflower. I know the chili oil is in the sauce. Ask any questions needed for clarification. Look at the image to get ideas of how it was made. Make you look at the image sent to make sure the final result will look like the image" # @param {type: "string", placeholder: "[your prompt]", isTemplate: true}

response = chat.send_message(
        Part.from_uri(file_uri=image_file_url, mime_type="image/png")
)


response = chat.send_message(
        Prompt
)


display(Markdown(response.text))


# **Expected Output**
After sending the prompt, the model will analyze the image and prompt, use Google Search to gather additional information from reviews and similar recipes, and then generate a recipe for the requested dish. It may also ask clarifying questions if it needs more details to create the most accurate recipe.

## **Answer questions or ask followup questions**

Since we setup a multi-turn chat, you are able to chat with Gemini and it remembers context from the earlier discussion.

In [ ]:
Prompt = "I would rather pan fry I am not comfortable with deep frying. " # @param {type: "string", placeholder: "[your prompt]", isTemplate: true}


response = chat.send_message(
        Prompt
)

display(Markdown(response.text))
